# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging 
import os 
import csv 
import pickle 
import requests 
import json 
import joblib

#from matplotlib import pyplot as plt
#import numpy as np
import pandas as pd 
#from sklearn import datasets 
import pkg_resources
from train import clean_data 

import azureml.core
from azureml.core.experiment import Experiment 
from azureml.core.workspace import Workspace 
from azureml.train.automl import AutoMLConfig 
from azureml.core.dataset import Dataset 
from azureml.widgets import RunDetails 
from azureml.pipeline.core import PipelineData, TrainingOutput
from azureml.pipeline.core import Pipeline 
from azureml.core.authentication import InteractiveLoginAuthentication 
from azureml.pipeline.steps import AutoMLStep 
from azureml.core.compute import AmlCompute 
from azureml.core.compute import ComputeTarget 
from azureml.core.compute_target import ComputeTargetException 
from azureml.core.model import InferenceConfig, Model 
from azureml.core.webservice import AciWebservice, Webservice 
from azureml.core.environment import Environment 
from azureml.core.run import Run 

#pip install pyarrow>=0.12.0 --upgrade

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

### Dataset and task

I am using the Titanic-dataset from Kaggle, that has information on the passengers of the Titanic. The data contains information on the passengers' age, siblings, sex, class on board, parch, fare etc. as can be seen below. I utilised the age, information on whether they were alone or with family, sex and fare to predict whether they survived or not. This is a classification task with a binary outcome.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
ws.write_config(path='.azureml')

# choose a name for experiment
experiment_name = 'Titanic_project'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

amlcompute_cluster_name = "ML-cluster-uda"

try:
      compute_target=ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
      print("Found existing cluster.Using it")
except ComputeTargetException:
      compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes=4)
      compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output = True, min_node_count=1, timeout_in_minutes=10)

Workspace name: quick-starts-ws-137178
Azure region: southcentralus
Subscription id: 9b72f9e6-56c5-4c16-991b-19c652994860
Resource group: aml-quickstarts-137178
Found existing cluster.Using it
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
found = False
key = "titanic"
description_text = "Titanic survival classification data from Kaggle"

if key in ws.datasets.keys(): 
        found = True
        data = ws.datasets[key] 
else :
    data = pd.read_csv("titanic.csv")

x, y = clean_data(data.to_pandas_dataframe())
x['y']=y
print(x.head(5))
print(x.describe())

# turning pandas dataframe to Tabular data
data_folder = './data'
os.makedirs(data_folder, exist_ok=True)
local_path = 'data/prepared.csv'
x.to_csv(local_path)

datastore = ws.get_default_datastore()
datastore.upload(src_dir='data', target_path='data')

dataset = Dataset.Tabular.from_delimited_files(path=[(datastore, ('data/prepared.csv'))])

   PassengerId  Pclass  Sex Age  SibSp  Parch     Fare  FamilySize  IsAlone  y
0            1       3    0   1      1      0   7.2500           1        0  0
1            2       1    1   2      1      0  71.2833           1        0  1
2            3       3    1   3      0      0   7.9250           0        1  1
3            4       1    1   4      1      0  53.1000           1        0  1
4            5       3    0   5      0      0   8.0500           0        1  0
       PassengerId      Pclass         Sex       SibSp       Parch  \
count   891.000000  891.000000  891.000000  891.000000  891.000000   
mean    446.000000    2.308642    0.352413    0.523008    0.381594   
std     257.353842    0.836071    0.477990    1.102743    0.806057   
min       1.000000    1.000000    0.000000    0.000000    0.000000   
25%     223.500000    2.000000    0.000000    0.000000    0.000000   
50%     446.000000    3.000000    0.000000    0.000000    0.000000   
75%     668.500000    3.000000    1.

## AutoML Configuration

TODO: Explain why you chose the automl settings and configuration you used below:
The settings I chose were designed to combine accuracy with also an idea of sparing resources. I chose a timeout of 20 minutes to make sure my experiments did not last too long. I chose the max concurrent iterations to 5 to limit the resources my experiments took. I chose the primary metric accuracy, because I could make it work for both automl and hyperdrive in the time I had. A better option would have been AUC. My task was a classification task. Early stopping as True is designed to save resources.

In [4]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="y",   
                             path = "./data",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             n_cross_validations=5,
                             **automl_settings
                            )


In [5]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on ML-cluster-uda with default configuration
Running on remote compute: ML-cluster-uda
Parent Run ID: AutoML_fa353ffa-1586-4468-9f9f-9cd78f2e9a1f

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing val

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

All the models received fairly similar results. The lowest accuracy was 0,6162 with run 25 utilising truncated SVCWrapper with XGBoostClassifier. Otherwise the models were fairly close to each other in accuracy. The VotingEnsemble often receives the best outcomes due to its ability to combine several different models and then they vote on the outcome therefore negating the possibly misleading results of any individual model.

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [7]:
best_run_auto, best_model = remote_run.get_output()
print(best_run_auto.get_tags())
print("best run auto:", best_run_auto)
print("best model:", best_model)

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


{'_aml_system_azureml.automlComponent': 'AutoML', '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":4,"CurrentNodeCount":4}', 'ensembled_iterations': '[24, 38, 1, 19, 15, 28, 21, 14, 11]', 'ensembled_algorithms': "['XGBoostClassifier', 'RandomForest', 'XGBoostClassifier', 'ExtremeRandomTrees', 'ExtremeRandomTrees', 'LightGBM', 'LightGBM', 'LightGBM', 'XGBoostClassifier']", 'ensemble_weights': '[0.07692307692307693, 0.3076923076923077, 0.15384615384615385, 0.07692307692307693, 0.07692307692307693, 0.07692307692307693, 0.07692307692307693, 0.07692307692307693, 0.07692307692307693]', 'best_individual_pipeline_score': '0.8024480572468772', 'best_individual_iteration': '24', '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True'}
best run auto: Run(Experiment: Titanic_project,
Id: AutoML_fa353ffa-1586-4468-9f9f-9cd78f2e9a1f_43,
Type: azureml.scriptrun,
Status: Completed)
best model: Pipeline(memory=None,
         steps=[(

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [8]:
description = 'Titanic survival AutoML model'
joblib.dump(value = best_model, filename = "titanic-survival-model.pkl")
model = Model.register(model_name='titanic-survival-model', 
                                     description = description, 
                                     model_path = 'titanic-survival-model.pkl', 
                                     tags={'area': 'titanic'}, 
                                     model_framework=Model.Framework.SCIKITLEARN, 
                                     model_framework_version='0.20.1',
                                     workspace = ws)

print(model)
print(model.name, model.id, model.version, sep=' --- ')

Registering model titanic-survival-model
Model(workspace=Workspace.create(name='quick-starts-ws-137178', subscription_id='9b72f9e6-56c5-4c16-991b-19c652994860', resource_group='aml-quickstarts-137178'), name=titanic-survival-model, id=titanic-survival-model:1, version=1, tags={'area': 'titanic'}, properties={})
titanic-survival-model --- titanic-survival-model:1 --- 1


In [9]:
# create environment
myenv = Environment.get(workspace=ws, name="AzureML-AutoML")
print(myenv.python.conda_dependencies.serialize_to_string())
myenv.save_to_file(path='output/', overwrite=False)

channels:
- anaconda
- conda-forge
- pytorch
dependencies:
- python=3.6.2
- pip=20.2.4
- pip:
  - azureml-core==1.21.0.post1
  - azureml-pipeline-core==1.21.0
  - azureml-telemetry==1.21.0
  - azureml-defaults==1.21.0
  - azureml-interpret==1.21.0
  - azureml-automl-core==1.21.0
  - azureml-automl-runtime==1.21.0
  - azureml-train-automl-client==1.21.0
  - azureml-train-automl-runtime==1.21.0.post1
  - azureml-dataset-runtime==1.21.0
  - inference-schema
  - py-cpuinfo==5.0.0
  - boto3==1.15.18
  - botocore==1.18.18
- numpy~=1.18.0
- scikit-learn==0.22.1
- pandas~=0.25.0
- py-xgboost<=0.90
- fbprophet==0.5
- holidays==0.9.11
- setuptools-git
- psutil>5.0.0,<6.0.0
name: azureml_7ade26eb614f97df8030bc480da59236



In [ ]:
# If one needs to add packages
#for pip_package in ["scikit-learn"]:
#    myenv.python.conda_dependencies.add_pip_package(pip_package)
#for pip_package in ["xgboost"]:
#    myenv.python.conda_dependencies.add_pip_package(pip_package)    

In [75]:
inference_config = InferenceConfig(entry_script="score.py",
                                   environment=myenv)

In [76]:
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1,
                                                       memory_gb = 1,
                                                      enable_app_insights=True) #auth_enabled = True - if using authentication

service = Model.deploy(
    workspace = ws, 
    name = 'titanic-deployment-service', 
    models = [model], 
    inference_config = inference_config, 
    deployment_config = deployment_config)

service.wait_for_deployment(True)

print(service.state)
print("scoring URI: " + service.scoring_uri)
print("swagger URI: " + service.swagger_uri)
print(service.get_logs())

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..........................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
scoring URI: http://bfb40653-d4ec-4bce-a686-ac4ab79fd0f0.southcentralus.azurecontainer.io/score
swagger URI: http://bfb40653-d4ec-4bce-a686-ac4ab79fd0f0.southcentralus.azurecontainer.io/swagger.json
2021-02-04T08:38:17,581946331+00:00 - nginx/run 
2021-02-04T08:38:17,582789059+00:00 - rsyslog/run 
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/

TODO: In the cell below, send a request to the web service you deployed to test it.

In [80]:
scoring_uri = 'http://bfb40653-d4ec-4bce-a686-ac4ab79fd0f0.southcentralus.azurecontainer.io/score'
# key = '2MoQ6XiUy9ejWNOATBnFIDrWnxtZGpQP'

# two sets of data to score
data = {"data":
        [
          {
            "Column1": 0,
            "PassengerId": 1709,
            "Pclass": 1,
            "Sex": 1,
            "Age": 25,
            "SibSp": 1,
            "Parch": 0,
            "Fare": 36.05,
            "FamilySize": 1,
            "IsAlone": 0
          }
      ]
    }
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)
    
# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
# headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print("Predicted:", resp.text, "Expected:0")

Predicted: [1] Expected:0


TODO: In the cell below, print the logs of the web service and delete the service

In [81]:
# standout suggestion - logging for the deployed service

logs = service.get_logs()

for line in logs.split('\n'):
    print(line)

2021-02-04T08:38:17,581946331+00:00 - nginx/run 
2021-02-04T08:38:17,582789059+00:00 - rsyslog/run 
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-02-04T08:38:17,694436858+00:00 - iot-server/run 
rsyslogd: /azureml-envs/azureml_7ade26eb614f97df8030bc480da5

In [82]:
# delete service
service.delete()